# NeuroFetal AI - Modular Training Pipeline

**Version 3.0** - Simplified & Modular

This notebook acts as a central launcher for the modular scripts in the codebase. It ensures consistency between local development and Cloud training.

### Steps:
1.  **Setup Environment**: Clone repo & install dependencies.
2.  **Data Ingestion**: Process raw PhysioNet data.
3.  **Train**: Run the deep learning training pipeline.
4.  **Evaluate**: Run ensemble and uncertainty metrics.
5.  **Serve**: Launch the dashboard (optional).

## 1. Setup Environment

In [ ]:
from getpass import getpass
import os

# 1. GitHub Authentication
GITHUB_REPO = "Krishna200608/NeuroFetal-AI"
print("Please enter your GitHub Personal Access Token (PAT):")
GITHUB_TOKEN = getpass()

os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
os.environ['GITHUB_REPO'] = GITHUB_REPO

In [ ]:
# 2. Clone Repository
import shutil

# Clean up any previous clone to avoid conflicts
if os.path.exists("/content/NeuroFetal-AI"):
    shutil.rmtree("/content/NeuroFetal-AI")

print("Cloning repository...")
!git clone https://{GITHUB_TOKEN}@github.com/{GITHUB_REPO}.git

# Set paths
os.chdir("/content/NeuroFetal-AI")
print("Cloned successfully!")

In [ ]:
# 3. Install Dependencies
print("Installing libraries...")
!pip install -q wfdb shap scipy imbalanced-learn pyngrok filterpy scikit-learn matplotlib seaborn pandas numpy tensorflow streamlit plotly python-dotenv
print("Dependencies installed.")

## 2. Data Ingestion
Processes raw `.dat`/`.hea` files into clean `.npy` arrays for training.

In [ ]:
# Run the data ingestion script
!python Code/scripts/data_ingestion.py

## 3. Training
Train the Tri-Modal Attention Fusion ResNet using 5-Fold Cross-Validation.
This script automatically handles:
*   Class Balancing (SMOTE)
*   Feature Extraction (CSP)
*   Model Checkpointing (saving best `.keras` files)

In [ ]:
# Run the main training script
!python Code/scripts/train.py

## 4. Advanced Evaluation
Generate metrics for:
1.  **Ensemble Performance**: Rank Averaging across folds (AUC maximization).
2.  **Uncertainty Quantification**: Monte Carlo Dropout confidence scores.

In [ ]:
print("Running Ensemble Evaluation (Rank Averaging)...")
!python Code/scripts/evaluate_ensemble.py

print("\nRunning Uncertainty Quantification...")
!python Code/scripts/evaluate_uncertainty.py

## 5. Launch Dashboard (Optional)
Run the Streamlit app directly from Colab using `ngrok`.
**Note**: You need an `NGROK_AUTH_TOKEN` set in your `.env` or pasted below.

In [ ]:
# Create a .env file locally for certain secrets if needed
auth_token = getpass("Enter Ngrok Auth Token (Validation optional if using .env): ")

if auth_token:
    with open("Code/.env", "w") as f:
        f.write(f"NGROK_AUTH_TOKEN={auth_token}\n")

print("Launching Streamlit App...")
!python Code/run_app.py